In [ ]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=23bbcc193b1e7f662a83ab331cec89024524506a8db4d3ba2cb24a66ee9a4fb7
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np

In [ ]:
'''
Baixando os dados da empresa NVIDIA (NVDA) no período de 2 de Julho de 2021 até
31 de Maio de 2024
'''
df = yf.download(tickers='NVDA', start='2023-05-31', end='2024-05-31')

[*********************100%%**********************]  1 of 1 completed


# 1. Análise Técnica

## 1.1 Indicadores de momentum

### 1.1.1 RSI - Relative Strenght Index

$RSI_t(n)$ = $100\frac{up_{t}(n)}{up_{t}(n)+down_{t}(n)}$

[Funcionamento do RSI](https://www.investopedia.com/terms/r/rsi.asp)

In [ ]:
from ta.momentum import RSIIndicator, PercentagePriceOscillator
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
rsi = RSIIndicator(df['Close'], window=30)
df['RSI'] = rsi.rsi()

In [ ]:
# Gráfico Preços / RSI

fig = make_subplots(rows=2, cols=1)
fig.add_trace(
    go.Scatter(name='RSI', x=df.index, y=df['RSI'], line=dict(color='red')),
    row=2, col=1
)
fig.add_trace(go.Candlestick(
    name='NVDA',
    x=df.index,
    close=df['Close'],
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    ),
row=1, col=1
)

# Desativando o zoom do gráfico
fig.update_layout(xaxis_rangeslider_visible=False)

# Ativando a figura
fig.show()

### 1.1.2 PPO - Porcentage Price Oscillator

$$
\text{PPO} = \left( \frac{\text{EMA}_{\text{curto}} - \text{EMA}_{\text{longo}}}{\text{EMA}_{\text{longo}}} \right) \times 100
$$

onde:
- $\text{EMA}_{\text{curto}}$ é a média móvel exponencial de curto prazo.
- $\text{EMA}_{\text{longo}}$ é a média móvel exponencial de longo prazo.

[Funcionamento do PPO](https://www.investopedia.com/terms/p/ppo.asp)

In [ ]:
ppo = PercentagePriceOscillator(df['Close'])
df['PPO'] = ppo.ppo()
df['PPO_hist'] = ppo.ppo_hist() # Histograma sobre intensidade dos volumes
df['PPO_signal'] = ppo.ppo_signal() # Sinal

In [ ]:
fig = make_subplots(rows=2,
                    cols=1,
                    specs=[[{"secondary_y" : True}],
                          [{"secondary_y": True}]
                          ])
fig.add_trace(go.Scatter(name='PPO', x=df.index, y=df['PPO'],
                         line=dict(color='red')), row=2, col=1)
fig.add_trace(go.Bar(name='PPO_hist',
              x=df.index,
              y=df['PPO_hist'],
              marker=dict(color='green')),
              row=2,
              col=1, secondary_y=True)
fig.add_trace(go.Scatter(name='PPO_signal',
              x=df.index,
              y=df['PPO_signal'],
              line=dict(color='blue')),
              row=2,
              col=1)
fig.add_trace(go.Candlestick(name='NVDA',
    x=df.index,
    close=df['Close'],
    open=df['Open'],
    high=df['High'],
    low=df['Low'],),
row=1, col=1
)

# Desativando o zoom do gráfico
fig.update_layout(xaxis_rangeslider_visible=False)

# Ativando a figura
fig.show()

## 1.2 Teoria de Dow aplicada em Python

[Entenda mais sobre a Teoria de Dow](https://www.nelogica.com.br/conhecimento/tutoriais/introtec/teoria-dow)